In [418]:
import pandas as pd
df_merge = pd.read_excel('data/olympic_final.xlsx')

In [419]:
df_merge.head()

,discipline_title,event_title,event_gender,participant_type,athlete_full_name,country_name,games_participations,athlete_year_birth,nb_gold,nb_silver,nb_bronze,Medals,game_end_date,game_start_date,game_location,game_season,game_year,city_location
0,Curling,Mixed Doubles,Mixed,GameTeam,Stefania CONSTANTINI,Italy,1,1999,1,0,0,1,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Winter,2022,Beijing
1,Curling,Mixed Doubles,Mixed,GameTeam,Almida DE VAL,Sweden,1,1997,0,0,1,1,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Winter,2022,Beijing
2,Curling,Mixed Doubles,Mixed,GameTeam,Oskar ERIKSSON,Sweden,4,1991,1,1,2,4,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Winter,2022,Beijing
3,Freestyle Skiing,Men's Freeski Big Air,Men,Athlete,Birk RUUD,Norway,1,2000,1,0,0,1,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Winter,2022,Beijing
4,Freestyle Skiing,Men's Freeski Big Air,Men,Athlete,Colby STEVENSON,USA,1,1997,0,1,0,1,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Winter,2022,Beijing


In [420]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


In [421]:
#Importation des bibliotheques
import pandas as pd
#from ydata_profiling import ProfileReport
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
import string
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, classification_report

In [422]:
# Sélection des colonnes pertinentes
features = df_merge[['discipline_title', 'event_title', 'event_gender', 'participant_type', 'country_name', 
                 'games_participations', 'athlete_year_birth', 'nb_gold', 'nb_silver', 'nb_bronze', 
                 'game_year', 'city_location']]
labels = df_merge['Medals']  # ici que nous voulons prédire le nombre de médailles

In [423]:
# Encodage des variables catégorielles
label_encoder = LabelEncoder()
categorical_columns = ['discipline_title', 'event_title', 'event_gender', 'participant_type', 'country_name', 'city_location']
for column in categorical_columns:
    features[column] = label_encoder.fit_transform(features[column])

C:\Users\tinad\AppData\Local\Temp\ipykernel_32368\4143448869.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[column] = label_encoder.fit_transform(features[column])
C:\Users\tinad\AppData\Local\Temp\ipykernel_32368\4143448869.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[column] = label_encoder.fit_transform(features[column])
C:\Users\tinad\AppData\Local\Temp\ipykernel_32368\4143448869.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [424]:
# Division des données en ensembles d'entraînement et de test
xtrain, xtest, ytrain, ytest = train_test_split(features, labels, test_size=0.20, random_state=42)

In [425]:
# Vérification des données
print(xtrain.head())
print(ytrain.head())

       discipline_title  event_title  event_gender  participant_type  \
10999                24         1087             2                 0   
4028                 62          489             3                 0   
13984                 4         1124             0                 0   
10497                 9          278             0                 0   
17293                 4          107             3                 0   

       country_name  games_participations  athlete_year_birth  nb_gold  \
10999            86                     7                1968        3   
4028              5                     3                1985        0   
13984           103                     1                1960        0   
10497            55                     2                1971        0   
17293             5                     3                1951        0   

       nb_silver  nb_bronze  game_year  city_location  
10999          5          1       1988             33  
4028      

In [426]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors =3)
knn.fit(xtrain,ytrain)

KNeighborsClassifier(n_neighbors=3)

In [427]:
#Prédiction
y_pred_knn = knn.predict(xtest)

In [428]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Evaluation
print('Accuracy KNN : {}%'.format(round(100 * accuracy_score(ytest, y_pred_knn), 2)))
print('Precision KNN : {}%'.format(round(100 * precision_score(ytest, y_pred_knn, average='macro'), 2)))
print('Recall KNN : {}%'.format(round(100 * recall_score(ytest, y_pred_knn, average='macro'), 2)))


Accuracy KNN : 61.99%
Precision KNN : 71.76%
Recall KNN : 66.47%


on a tester des n_neighbors de 3 a 9 et la valeur de neighbors qui donne les meilleurs résultats est 3


In [429]:
#Random forest modèle
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(xtrain, ytrain)

RandomForestClassifier(random_state=42)

In [430]:
#Prédiction
random_forest_pred = rf_classifier.predict(xtest)

In [431]:

# Evaluation
print('Accuracy random forest  : {}%'.format(round(100 * accuracy_score(ytest, random_forest_pred), 2)))
print('Precision random forest : {}%'.format(round(100 * precision_score(ytest, random_forest_pred, average='macro'), 2)))
print('Recall random forest : {}%'.format(round(100 * recall_score(ytest, random_forest_pred, average='macro'), 2)))


Accuracy random forest  : 99.88%
Precision random forest : 99.74%
Recall random forest : 99.72%


IA - Prédire le nombre de médailles (TOTAL, OR, ARGENT, BRONZE) en fonction du pays 

In [432]:
# colonnes pertinentes pour la prédiction des médailles
features = df_merge[['country_name', 'games_participations', 'athlete_year_birth']]
labels = df_merge[['nb_gold', 'nb_silver', 'nb_bronze', 'Medals']]

In [433]:
features.head()

,country_name,games_participations,athlete_year_birth
0,Italy,1,1999
1,Sweden,1,1997
2,Sweden,4,1991
3,Norway,1,2000
4,USA,1,1997


In [434]:

labels.head()

,nb_gold,nb_silver,nb_bronze,Medals
0,1,0,0,1
1,0,0,1,1
2,1,1,2,4
3,1,0,0,1
4,0,1,0,1


In [435]:
#features.groupby(['country_name']).sum().reset_index()

In [436]:



# Encodage des variables catégorielles
label_encoder = LabelEncoder()
features['country_name'] = label_encoder.fit_transform(features['country_name'])

# Division des données en ensembles d'entraînement et de test
xtrain, xtest, ytrain, ytest = train_test_split(features, labels, test_size=0.20, random_state=42)

# Vérification des données
print(xtrain.head())
print(ytrain.head())


       country_name  games_participations  athlete_year_birth
10999            86                     7                1968
4028              5                     3                1985
13984           103                     1                1960
10497            55                     2                1971
17293             5                     3                1951
       nb_gold  nb_silver  nb_bronze  Medals
10999        3          5          1       9
4028         0          0          1       1
13984        0          1          0       1
10497        0          0          1       1
17293        0          3          0       3


C:\Users\tinad\AppData\Local\Temp\ipykernel_32368\1430735612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['country_name'] = label_encoder.fit_transform(features['country_name'])


In [437]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

# Modélisation avec Random Forest
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))

# Entraînement du modèle
model.fit(xtrain, ytrain)

# Prédictions
y_pred = model.predict(xtest)

# Évaluation du modèle
mse = mean_squared_error(ytest, y_pred, multioutput='raw_values')
print('Mean Squared Error for nb_gold:', mse[0])
print('Mean Squared Error for nb_silver:', mse[1])
print('Mean Squared Error for nb_bronze:', mse[2])
print('Mean Squared Error for Medals:', mse[3])


Mean Squared Error for nb_gold: 0.7098964161280625
Mean Squared Error for nb_silver: 0.3729059197266408
Mean Squared Error for nb_bronze: 0.33311795025948154
Mean Squared Error for Medals: 1.2317766761728506


In [438]:


# Faire des prédictions
predictions = model.predict(features)



In [439]:
# Décoder les noms des pays à partir des valeurs encodées
decoded_countries = label_encoder.inverse_transform(new_data['country_name'])

# Afficher les prédictions avec les noms des pays
print("Prédictions de médailles pour chaque pays :")
for i, country in enumerate(decoded_countries):
    print(f"Pays : {country}, Médailles prédites : {predictions[i]}")


Prédictions de médailles pour chaque pays :
Pays : Italy, Médailles prédites : [0.63 0.13 0.28 1.  ]
Pays : Sweden, Médailles prédites : [0.07104762 0.15144444 0.47766667 0.70134921]
Pays : Sweden, Médailles prédites : [0.86 1.14 1.59 3.65]
Pays : Norway, Médailles prédites : [0.965      0.34680952 0.04       1.36180952]
Pays : USA, Médailles prédites : [0.12935361 0.6283953  0.24225109 1.        ]
Pays : Switzerland, Médailles prédites : [0.58666667 0.401      0.04       1.08      ]
Pays : China, Médailles prédites : [1.94666667 0.96       0.03       2.93416667]
Pays : China, Médailles prédites : [1.94666667 0.96       0.03       2.93416667]
Pays : China, Médailles prédites : [1.94666667 0.96       0.03       2.93416667]
Pays : Russia, Médailles prédites : [0.135      0.249      0.78666667 1.03      ]
Pays : Germany, Médailles prédites : [0.01875    0.50490476 0.48528571 1.01066667]
Pays : Belarus, Médailles prédites : [0.022  0.4955 0.4945 1.03  ]
Pays : USA, Médailles prédites : [0.

In [441]:
# DataFrame pour stocker les prédictions avec les noms des pays décodés
predictions_df = pd.DataFrame({
    'country_name': decoded_countries,  # Décodage des noms des pays
    'nb_gold_pred': predictions[:, 0],  # Prédictions pour le nombre d'or
    'nb_silver_pred': predictions[:, 1],  # Prédictions pour le nombre d'argent
    'nb_bronze_pred': predictions[:, 2],  # Prédictions pour le nombre de bronze
    'Medals_pred': predictions[:, 3]  # Prédictions pour le nombre total de médailles
})


# Trier les pays par nombre total de médailles prédites
top_10_countries = predictions_df.sort_values(by='Medals_pred', ascending=False).head(10)

# Afficher le top 10 des pays prédits
print("Top 10 des pays prédits pour remporter le plus grand nombre de médailles :")
print(top_10_countries[['country_name', 'Medals_pred']])


Top 10 des pays prédits pour remporter le plus grand nombre de médailles :
      country_name  Medals_pred
20037       Russia    15.243666
20041       Russia    15.243666
20035       Russia    15.243666
20034       Russia    15.243666
20033       Russia    15.243666
20032       Russia    15.243666
20031       Russia    15.243666
20030       Russia    15.243666
20028       Russia    15.243666
20027       Russia    15.243666
